Load all packages you need.

In [1]:
import pandas as pd

Read the data.

In [ ]:
df = pd.read_csv("/Users/davidsuss/Documents/GitHub/Applied-Analytics/data/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Let's deal with the missing data points in "BloodPressure". <br>
First remove the outlier from the last coding break.<br>
Create a copy of the outlier-filtered data frame and impute the missing values with the "BloodPressure" median value.

In [ ]:
import pandas as pd

# Zuerst alle 0-Werte in "BloodPressure" mit dem Median ersetzen
median_bp = df["BloodPressure"].replace(0, pd.NA).median()
df["BloodPressure"] = df["BloodPressure"].replace(0, median_bp)

# Berechnung der Tukey's Fence Grenzen nach der Median-Imputation
Q1 = df["BloodPressure"].quantile(0.25)
Q3 = df["BloodPressure"].quantile(0.75)
IQR = Q3 - Q1

lower_bound_tukey = Q1 - 1.5 * IQR
upper_bound_tukey = Q3 + 1.5 * IQR

# Bereinigung: Entfernen der Outlier basierend auf Tukey's Fence
df_cleaned = df[
    (df["BloodPressure"] >= lower_bound_tukey) & 
    (df["BloodPressure"] <= upper_bound_tukey)
].copy()

print(df_cleaned.shape[0])

754


Create a second copy of the outlier-filtered data frame. <br>
Perform a hot-deck/distribution imputation. First, estimate the distribution parameters of "BloodPressure". Then, sample the values used for imputation.

In [21]:
import numpy as np

# Erstelle eine zweite Kopie des bereinigten DataFrames
df_hotdeck = df_cleaned.copy()

# Schätze die Verteilungsparameter (Mittelwert und Standardabweichung) von "BloodPressure"
mean_bp = df_hotdeck["BloodPressure"].mean()
std_bp = df_hotdeck["BloodPressure"].std()

# Identifiziere die Zeilen, in denen "BloodPressure" gleich 0 ist
zero_bp_indices = df_hotdeck["BloodPressure"] == 0

# Ziehe Zufallswerte aus einer Normalverteilung basierend auf den geschätzten Parametern
imputed_values = np.random.normal(loc=mean_bp, scale=std_bp, size=zero_bp_indices.sum())

# Weise die imputierten Werte den 0-Positionen zu
df_hotdeck.loc[zero_bp_indices, "BloodPressure"] = imputed_values

print(df_hotdeck.head())
print(df_hotdeck.shape[0])

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
754


Create a third copy of the outlier-filtered data frame. <br>
Perform a regression-based imputation. First, choose 3 variables that are correlated with "BloodPressure" (Hint, create a correlation matrix or have a look at the "Associations"-Button in sweetviz).<br>
Second, fit a linear regression model with those three variables as predictors. Let's ignore for the moment that the other variables might also include outlier and missing data.<br>
Now, use the regression model to estimate the imputed value for each missing data point.

Now plot the original data next to the imputed data. Inspect the differences!